In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
DEBUG = False

In [ ]:
%run ./nb_core/root_dirs.ipynb
setup_syspath_disentangle(DEBUG)
%run ./nb_core/disentangle_imports.ipynb

In [ ]:
from disentangle.analysis.plot_utils import clean_ax
from disentangle.core.tiff_reader import load_tiff
from disentangle.config_utils import load_config, get_configdir_from_saved_predictionfile
from disentangle.core.data_split_type import DataSplitType
from disentangle.scripts.evaluate import get_highsnr_data
import ml_collections

In [ ]:
noise_levels = [8900]
pred_dir = '/group/jug/ashesh/data/paper_stats/'

usplit_fname = {4450: 'Test_P64_G32_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_42.tif',
                8900: 'Test_P64_G32_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_43.tif'}

denoiSplit_fname = {4450: 'Test_P256_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_16.tif',
                    8900: 'Test_P256_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_15.tif'}

denoiSplitNM_fname = {4450: 'Test_P256_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_62.tif',
                      8900: 'Test_P256_G64_M5_Sk44/pred_disentangle_2402_D16-M3-S0-L0_61.tif'}
hdn_usplit = {} #{4450: 'pred_disentangle_2402_D23-M3-S0-L0_34.tif'}

In [ ]:
def sanity_check_config():
    data_dicts = [usplit_fname, denoiSplit_fname, denoiSplitNM_fname]
    for ith_data, ddict in enumerate(data_dicts):
        for noise,fname in ddict.items():
            configdir = get_configdir_from_saved_predictionfile(fname)
            config = load_config(configdir)
            assert 'synthetic_gaussian_scale' in config.data
            assert config.data.synthetic_gaussian_scale == noise, f'{ith_data} {fname}: noise: {noise}, config: {config.data.synthetic_gaussian_scale}'
    


In [ ]:
sanity_check_config()

### Loading target

In [ ]:
configdir  = get_configdir_from_saved_predictionfile(denoiSplitNM_fname[noise_levels[0]])
config = ml_collections.ConfigDict(load_config(configdir))
highsnr_data = get_highsnr_data(config, config.datadir, DataSplitType.Test)

### Loading predictions

In [ ]:
import json
usplit_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in usplit_fname.items()}
denoiSplit_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in denoiSplit_fname.items()}
denoiSplitNM_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in denoiSplitNM_fname.items()}
hdn_usplit_data = {k: load_tiff(os.path.join(pred_dir, v)) for k,v in hdn_usplit.items()}

# Undoing the offset.
for k,v in usplit_fname.items():
     with open(os.path.join(pred_dir, v.replace('.tif', '.json')),'rb') as f:
        offset = float(json.load(f)['offset'])
        usplit_data[k] = usplit_data[k] + offset

for k,v in denoiSplit_fname.items():
     with open(os.path.join(pred_dir, v.replace('.tif', '.json')),'rb') as f:
        offset = float(json.load(f)['offset'])
        denoiSplit_data[k] = denoiSplit_data[k] + offset

for k,v in denoiSplitNM_fname.items():
    with open(os.path.join(pred_dir, v.replace('.tif', '.json')),'rb') as f:
        offset = float(json.load(f)['offset'])
        denoiSplitNM_data[k] = denoiSplitNM_data[k] + offset


### Cropping the target to get to the same shape as the predictions

In [ ]:
shape = usplit_data[noise_levels[0]].shape
highsnr_data = highsnr_data[:, :shape[1], :shape[2]].copy()

In [ ]:
def sanity_check_data():
    # all shapes should be same
    for noise_level in noise_levels:
        shape = usplit_data[noise_level].shape
        if noise_level in denoiSplit_data:
            assert shape == denoiSplit_data[noise_level].shape
        if noise_level in denoiSplitNM_data:
            assert shape == denoiSplitNM_data[noise_level].shape
        if noise_level in hdn_usplit_data:
            assert shape == hdn_usplit_data[noise_level].shape
        assert shape == highsnr_data.shape
            

In [ ]:
sanity_check_data()

In [ ]:
import numpy as np
def get_noisy_data(noise_level):
    return highsnr_data + np.random.normal(0, noise_level, highsnr_data.shape)

In [ ]:
fig_w

In [ ]:
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
from disentangle.analysis.plot_utils import add_pixel_kde, clean_ax
from disentangle.core.psnr import RangeInvariantPsnr

#### inset specific
inset_rect=[0.1, 0.1, 0.4, 0.2]
inset_min_labelsize=10
color_ch_list=['goldenrod', 'cyan']
color_pred='red'
insetplot_xmax_value = None
insetplot_xmin_value = None
plt_dsample = 4
####
data_idx = 0
img_sz = 3
ncol_imgs = 5
nrow_imgs = 2
example_spacing = 1
grid_factor = 5
nimgs = 1
noise_level = 8900
# extra spacing for c0. It does not work. Don't know why. I think there is some integer division happening.
c0_extra = 1

noisy_data = get_noisy_data(noise_level)

# for subscripts and superscripts
params = {'mathtext.default': 'regular' }          
plt.rcParams.update(params)

def get_psnr_str(prediction, ch_idx):
    return f'{RangeInvariantPsnr(highsnr_data[data_idx,...,ch_idx][None], prediction[data_idx,...,ch_idx][None]).item():.1f}' 

def add_psnr_str(ax_, psnr):
    """
    Add psnr string to the axes
    """
    textstr = f'PSNR\n{psnr}'
    props = dict(
        boxstyle='round', 
        facecolor='gray', alpha=0.3)
    # place a text box in upper left in axes coords
    ax_.text(0.05, 0.95, textstr, transform=ax_.transAxes, fontsize=11,
            verticalalignment='top', bbox=props, color='white')


fig_w = ncol_imgs * img_sz + c0_extra/grid_factor
fig_h = int(img_sz * nrow_imgs + (example_spacing * (nimgs - 1)) / grid_factor )
fig = plt.figure(figsize=(fig_w, fig_h))
gs = GridSpec(nrows=int(grid_factor * fig_h), ncols=int(grid_factor * fig_w), hspace=0.2, wspace=0.2)
grid_img_sz = img_sz * grid_factor

# input
ax_temp = fig.add_subplot(gs[:grid_img_sz,:grid_img_sz])
ax_temp.imshow(np.mean(noisy_data[data_idx], axis=-1), cmap='magma')
legend_ax = ax_temp

clean_ax(ax_temp)

# ax[0,0].set_title('Input')
ax_temp = fig.add_subplot(gs[:grid_img_sz, (c0_extra+grid_img_sz):(c0_extra + grid_img_sz * 2)])
ax_temp.imshow(noisy_data[data_idx,:,:,0], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  noisy_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  inset_min_labelsize,
                                  label1='NoisyCh1',
                                  label2='Ch1',
                                  plot1_kwargs={'linestyle':'--'},
                                  color1=color_ch_list[0],
                                  color2=color_ch_list[0],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2, c0_extra+grid_img_sz:c0_extra + grid_img_sz * 2])
ax_temp.imshow(noisy_data[data_idx,:,:,1], cmap='magma', vmin=highsnr_data[data_idx,...,1].min(), vmax=highsnr_data[data_idx,...,1].max())
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  noisy_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  inset_min_labelsize,
                                  label1='NoisyCh2',
                                  label2='Ch2',
                                  color1=color_ch_list[1],
                                  plot1_kwargs={'linestyle':'--'},
                                  color2=color_ch_list[1],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[:grid_img_sz, c0_extra+grid_img_sz * 2:c0_extra+grid_img_sz * 3])
ax_temp.imshow(usplit_data[noise_level][data_idx,...,0], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  usplit_data[noise_level][data_idx,::plt_dsample,::plt_dsample,0],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  inset_min_labelsize,
                                  label1='Pred1',
                                  label2='Ch1',
                                  color1=color_pred,
                                  color2=color_ch_list[0],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)
add_psnr_str(ax_temp, get_psnr_str(usplit_data[noise_level], 0))
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2,c0_extra+grid_img_sz * 2:c0_extra+grid_img_sz * 3])
ax_temp.imshow(usplit_data[noise_level][data_idx,...,1], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  usplit_data[noise_level][data_idx,::plt_dsample,::plt_dsample,1],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  inset_min_labelsize,
                                  label1='Pred2',
                                  label2='Ch2',
                                  color1=color_pred,
                                  color2=color_ch_list[1],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

add_psnr_str(ax_temp, get_psnr_str(usplit_data[noise_level], 1))
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[:grid_img_sz, c0_extra+grid_img_sz * 3:c0_extra+grid_img_sz * 4])
ax_temp.imshow(denoiSplitNM_data[noise_level][data_idx,...,0], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  denoiSplitNM_data[noise_level][data_idx,::plt_dsample,::plt_dsample,0],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  inset_min_labelsize,
                                  label1='Pred1',
                                  label2='Ch1',
                                  color1=color_pred,
                                  color2=color_ch_list[0],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

add_psnr_str(ax_temp, get_psnr_str(denoiSplitNM_data[noise_level], 0))
clean_ax(ax_temp)
ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2, c0_extra+grid_img_sz * 3:c0_extra+grid_img_sz * 4])
ax_temp.imshow(denoiSplitNM_data[noise_level][data_idx,...,1], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  denoiSplitNM_data[noise_level][data_idx,::plt_dsample,::plt_dsample,1],
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  inset_min_labelsize,
                                  label1='Pred2',
                                  label2='Ch2',
                                  color1=color_pred,
                                  color2=color_ch_list[1],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

add_psnr_str(ax_temp, get_psnr_str(denoiSplitNM_data[noise_level], 1))
clean_ax(ax_temp)

ax_temp = fig.add_subplot(gs[:grid_img_sz, c0_extra+grid_img_sz * 4:c0_extra+grid_img_sz * 5])
ax_temp.imshow(highsnr_data[data_idx,...,0], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,0],
                                  None,
                                  inset_min_labelsize,
                                  label1='Ch1',
                                  color1=color_ch_list[0],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

clean_ax(ax_temp)


ax_temp = fig.add_subplot(gs[grid_img_sz:grid_img_sz * 2, c0_extra+grid_img_sz * 4:c0_extra+grid_img_sz * 5])
ax_temp.imshow(highsnr_data[data_idx,...,1], cmap='magma')
_ = add_pixel_kde(ax_temp,
                                  inset_rect,
                                  highsnr_data[data_idx,::plt_dsample,::plt_dsample,1],
                                  None,
                                  inset_min_labelsize,
                                  label1='Ch2',
                                  color1=color_ch_list[1],
                                  plot_xmax_value=insetplot_xmax_value,
                                  plot_xmin_value=insetplot_xmin_value)

clean_ax(ax_temp)

# manually setting legends
import matplotlib.lines as mlines
line_ch1 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[0], linestyle='-', label='$C_1$')
line_ch2 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[1], linestyle='-', label='$C_2$')
line_pred = mlines.Line2D([0, 1], [0, 1], color=color_pred, linestyle='-', label='Pred')
line_noisych1 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[0], linestyle='--', label='$C^N_1$')
line_noisych2 = mlines.Line2D([0, 1], [0, 1], color=color_ch_list[1], linestyle='--', label='$C^N_2$')

legend = legend_ax.legend(handles=[line_ch1, line_noisych1, line_ch2, line_noisych2, line_pred], loc='upper left', frameon=False, labelcolor='white', 
                          prop={'size': 11})

In [ ]:
# idx_list = [58,61,65, 30,31,33, 34,29,63, 32,35,36, 48,42,50,51,54,59, 55,43,49, 68,56,62,74,80,78, 75,79,82, 76,81,83, 77,85, 84]
# for idx in idx_list:
#     data  = load_tiff(f'/group/jug/ashesh/data/paper_stats/All_P128_G64_M50_Sk44/pred_disentangle_2402_D16-M23-S0-L0_{idx}.tif')
#     r0 = np.logical_and(data > 64000, data < 64500)
#     r1 = np.logical_and(data > 64500, data < 65000)
#     r2 = data > 65000
#     print(idx, data.min(), data.max(), r0.sum(), r1.sum(), r2.sum()) 

In [ ]:
a = np.uint16([65535]) + 10
a